# Classificação de Texto utilizando CountVectorizer
Autor: Lucas Kenji Sudo

E-mail: lucaskenjis@gmail.com

Linkedin : linkedin.com/in/lucas-sudo-701647121/

In [1]:
import pandas as pd
import nltk
import re
import numpy as np
import time

# Imports para NLP
#from nltk import word_tokenize
from bs4 import BeautifulSoup
from nltk import regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

start_time = time.time()

In [2]:
df = pd.read_csv('IMDB Dataset.csv')

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,negative
freq,5,25000


#### Remover review duplicadas

In [5]:
df = df[~df.review.duplicated()]

#### Remover colunas null

In [6]:
df=df[~df.review.isnull()]

#### Limpeza

Método para limpeza de texto aplicando:
    - replace para um caractere específico
    - isdigit para verificar se a palavra é um digito e assim, remove-la.
    - limpeza de tag html usando BeautifulSoup
    - regexp_tokenize para transformar textos em tokens, e coletar apenas palavras através do regex [\w]+. Ignorando acentos.
    - .lower() para deixar as palavras minusculas
    - stopwords para remover palavras que não agregam informações
    - stemming para extrair o radical das palavras

In [7]:
#nltk.download('stopwords')
ps = PorterStemmer()
eng_stpw = set(stopwords.words('english'))

def padronizardados(text):
    # remover caractere especifico
    text = text.replace('\ ','')
    # remover digitos
    text = ''.join([i for i in text if not i.isdigit()])
    # remover tags html
    soup = BeautifulSoup(text, "html.parser")
    #tokenization e lower case
    text = regexp_tokenize(soup.get_text().lower(),"[\w]+")
    #remover stopwords
    temp = []
    for t in text:
        if t not in eng_stpw:
            #aplicar o stemming
            temp.append(ps.stem(t))
    new_text = (' '.join(temp))
    
    return new_text

##### Texto antes do método

In [8]:
df.review[30]

'Taut and organically gripping, Edward Dmytryk\'s Crossfire is a distinctive suspense thriller, an unlikely "message" movie using the look and devices of the noir cycle.<br /><br />Bivouacked in Washington, DC, a company of soldiers cope with their restlessness by hanging out in bars. Three of them end up at a stranger\'s apartment where Robert Ryan, drunk and belligerent, beats their host (Sam Levene) to death because he happens to be Jewish. Police detective Robert Young investigates with the help of Robert Mitchum, who\'s assigned to Ryan\'s outfit. Suspicion falls on the second of the three (George Cooper), who has vanished. Ryan slays the third buddy (Steve Brodie) to insure his silence before Young closes in.<br /><br />Abetted by a superior script by John Paxton, Dmytryk draws precise performances from his three starring Bobs. Ryan, naturally, does his prototypical Angry White Male (and to the hilt), while Mitchum underplays with his characteristic alert nonchalance (his role, h

##### Aplicando método de limpeza aos textos

In [9]:
df.review = df.review.apply(padronizardados)

##### Texto depois do método

In [10]:
df.review[30]

'taut organ grip edward dmytryk crossfir distinct suspens thriller unlik messag movi use look devic noir cycl bivouack washington dc compani soldier cope restless hang bar three end stranger apart robert ryan drunk belliger beat host sam leven death happen jewish polic detect robert young investig help robert mitchum assign ryan outfit suspicion fall second three georg cooper vanish ryan slay third buddi steve brodi insur silenc young close abet superior script john paxton dmytryk draw precis perform three star bob ryan natur prototyp angri white male hilt mitchum underplay characterist alert nonchal role howev central young may never better gloria graham give first fulli fledg rendit smart mouth vulner tramp sad sack leech life paul kelli haunt us small peripher role make memor polit engag dmytryk perhap inevit succumb sermon pretti much confin young reminisc irish grandfath die hand bigot centuri earlier thu incident stretch chronolog limit least attempt render explan howev glib ryan

#### Transformar sentiments em 0 e 1 (Label encoder)

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
tag = le.fit_transform(df.sentiment)
tag

array([1, 1, 1, ..., 0, 0, 0])

# CountVectorizer
   Ao construir modelos para análise de texto não podemos trabalhar diretamente com os dados na forma textual, pois o computador trabalha com dados na forma numérica. Para analisar dados de texto é preciso separar o texto em palavras ou em conjunto de palavras, através do processo de tokenization, e transformar essas palavras em representação numérica, através do processo de extração de características (features).
    Nesse jupyter notebook irei utilizar a técnica CountVectorizer.
    https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html 

   O conceito deste método é contar as vezes em que a palavra aparece no documento e usa esse valor como atributo (maior frequência = maior peso). 
   O problema dessa abordagem é que, a frequência que uma palavra aparece não necessariamente a torna mais relevante. Caso não seja feito um pré-processamento do texto, palavras que não carregam informações relevantes como ‘a’,’the’,’an’,’o’ aparecerão com mais frequência em relação a palavras com informações relevantes, prejudicando o modelo.

In [12]:
cv=CountVectorizer()
corpus = ['Esse filme é bom','Esse filme é ótimo']

x = cv.fit(corpus)
#Vocabulario com os id inteiros fixos
print(x.vocabulary_) 
# Features
print(cv.get_feature_names())

{'esse': 1, 'filme': 2, 'bom': 0, 'ótimo': 3}
['bom', 'esse', 'filme', 'ótimo']


In [13]:
y = cv.transform(corpus)
print('shape:',y.shape,'\n')
print('array,vocab   freq')
print(y)
print(y.toarray())

shape: (2, 4) 

array,vocab   freq
  (0, 0)	1
  (0, 1)	1
  (0, 2)	1
  (1, 1)	1
  (1, 2)	1
  (1, 3)	1
[[1 1 1 0]
 [0 1 1 1]]


##### Split dados
    - Optei por separar os dados em treino, validação e teste

In [12]:
from sklearn.model_selection import train_test_split
  
X_train, X_test, y_train, y_test  = train_test_split(df.review, df.sentiment, test_size=0.2, random_state=1)

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1)

#### Bag of words utilizando ngram no range(1,2) para procurar palavras que se complementam

In [13]:
vectorizer = CountVectorizer(ngram_range=(1, 2),max_features=10000)
X_train = vectorizer.fit_transform(X_train)
X_val = vectorizer.transform(X_val)
X_test = vectorizer.transform(X_test)

In [14]:
vectorizer.get_feature_names()[0:10]

['aaron',
 'abandon',
 'abbott',
 'abc',
 'abduct',
 'abil',
 'abl',
 'abl get',
 'abl make',
 'abl see']

In [28]:
#vectorizer.vocabulary_        #printar todo o vocabulário
vectorizer.vocabulary_['like'] #printar o ID da palavra like 

4826

In [29]:
print(X_train[30])

  (0, 6705)	1
  (0, 7330)	1
  (0, 864)	1
  (0, 4508)	1
  (0, 5956)	1
  (0, 1560)	1
  (0, 4036)	1
  (0, 8800)	1
  (0, 9602)	1
  (0, 9027)	1
  (0, 4016)	1
  (0, 9173)	1
  (0, 2478)	1
  (0, 2313)	1
  (0, 1921)	1
  (0, 2780)	1
  (0, 7053)	1
  (0, 3805)	1
  (0, 7993)	1
  (0, 5548)	1
  (0, 7016)	1
  (0, 3448)	1
  (0, 8667)	1
  (0, 7174)	1
  (0, 6476)	1
  :	:
  (0, 6763)	1
  (0, 5316)	1
  (0, 9798)	1
  (0, 9117)	1
  (0, 348)	1
  (0, 9177)	1
  (0, 6337)	2
  (0, 3237)	1
  (0, 9493)	1
  (0, 8850)	1
  (0, 861)	1
  (0, 8212)	1
  (0, 1915)	1
  (0, 1580)	1
  (0, 4564)	1
  (0, 4899)	1
  (0, 7741)	1
  (0, 752)	1
  (0, 2917)	1
  (0, 6695)	1
  (0, 1269)	1
  (0, 2651)	1
  (0, 8028)	1
  (0, 4826)	2
  (0, 7597)	1


#### Feature selection + L1 e Regressao logistica
A seleção de atributos ( Feature Selection) é uma técnica responsável por selecionar os atributos mais relevantes/importantes para a criação do modelo. Sua utilização trás vantagens como: 
   - Minimizar o problema de overfitting sem perder muita informação.
   - Diminuir o tempo de treinamento.
   
Para seleção de features foi utilizado a regularização L1, também conhecida como Lasso regression. O L1 penaliza os coeficientes com um alto grau de correlação entra si jogando seu valor a ZERO, ou seja, removendo a feature e diminuindo a complexidade do modelo.

In [17]:
cs=[.01,.1,1,10,100]
#
summary=[]

for c in cs:
    
    #seleção de atributos
    logreg = LogisticRegression(solver='saga', penalty='l1',C=c, max_iter=100).fit(X_train, y_train)
    select_features = SelectFromModel(logreg, prefit=True)
    
    X_train_sel=select_features.transform(X_train)
    X_test_sel=select_features.transform(X_test)
    X_val_sel=select_features.transform(X_val)

    #fittando o modelo
    model_cv = LogisticRegression(solver='saga', max_iter=100).fit(X_train_sel, y_train)
    
    #avaliando acurácia
    model_cv_score = model_cv.score(X_val_sel, y_val)
    
    #resumo da validação
    summary.append((c,np.shape(X_train_sel)[1],model_cv_score))
    
    print(round((time.time() - start_time)/60,2),"minutos \n")

C:\Users\lucaskenjis\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


9.67 minutos 

10.33 minutos 

12.71 minutos 

15.5 minutos 

18.32 minutos 



selected_feat = X_train.columns[(select_features.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(np.sum(sel_.estimator_.coef_ == 0)))

In [18]:
for i in summary:
  print("C=%.2f Features=%d Acc=%3.4f" %i)

C=0.01 Features=205 Acc=0.8560
C=0.10 Features=1818 Acc=0.8933
C=1.00 Features=8524 Acc=0.8968
C=10.00 Features=9973 Acc=0.8974
C=100.00 Features=9999 Acc=0.8974


#### Criação do modelo com o melhor peso C

In [19]:
logreg = LogisticRegression(solver='saga', penalty='l1',C=10, max_iter=100).fit(X_train, y_train)
select_features = SelectFromModel(logreg, prefit=True)
    
X_train_sel=select_features.transform(X_train)
X_test_sel=select_features.transform(X_test)
X_val_sel=select_features.transform(X_val)

model_cv = LogisticRegression(solver='saga', max_iter=100).fit(X_train_sel, y_train)
    
lg_score = model_cv.score(X_val_sel, y_val)
    
summary.append((c,np.shape(X_train_sel)[1],lg_score))
    
print(round((time.time() - start_time)/60,2),"minutos \n")

71.07 minutos 



#### Testando o modelo criado com as variáveis teste

In [20]:
print("Acurácia na base de teste=%3.3f \n" % model_cv.score(X_test_sel, y_test))

y_pred = model_cv.predict(X_test_sel)
print(confusion_matrix(y_pred,y_test))

Acurácia na base de teste=0.884 

[[2167  248]
 [ 329 2214]]
